# 1 · bHLH anchors (PF00010) — ASC project

**Goal:** scan full-length ASC proteins (`ASC_targets.fasta`) with `PF00010.hmm` (bHLH) to detect **all bHLH domains per protein**, derive anchor positions (domain centers), and export clean TSV/GFF/JSON for downstream motif analysis (MEME) and tree overlays.

**Notes**
- We analyze **whole protein sequences**, not only C‑terminal regions.
- Multiple bHLH domains per protein are supported.
- Works with `hmmsearch` (preferred) or falls back to `hmmscan` if needed.
- Input files are expected in `data/` as set by the `0_setup` notebook/script.



In [ ]:
import os, shutil, subprocess, sys, json
from pathlib import Path
import pandas as pd
from datetime import datetime

#  Project paths 
PROJ = Path.cwd().resolve().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA = PROJ / "data"
OUT  = PROJ / "results"
CLADES = OUT / "clades"
MOTIFS = OUT / "motifs"
REPORTS = OUT / "reports"

for d in (DATA, OUT, CLADES, MOTIFS, REPORTS):
    d.mkdir(parents=True, exist_ok=True)

# inputs
IN_TARGETS = DATA / "ASC_targets.fasta"     # proteins, ungapped
HMM_PF00010 = DATA / "PF00010.hmm"          # pressed already (h3*)
PFAM_A = DATA / "Pfam-A.hmm"                # optional; not required here

print("Project:", PROJ)
print("Data dir:", DATA)
print("Targets:", IN_TARGETS, "exists:", IN_TARGETS.exists())
print("PF00010.hmm:", HMM_PF00010, "exists:", HMM_PF00010.exists())


Project: /Users/gorkemdurmaz/Desktop/asc_project_10
Data dir: /Users/gorkemdurmaz/Desktop/asc_project_10/data
Targets: /Users/gorkemdurmaz/Desktop/asc_project_10/data/ASC_targets.fasta exists: True
PF00010.hmm: /Users/gorkemdurmaz/Desktop/asc_project_10/data/PF00010.hmm exists: True


## Parameters
- `EVALUE_SEQ_MAX`: maximum full-sequence E-value to keep a hit.
- `EVALUE_DOM_MAX`: maximum domain (independent) E-value to keep a domain.
- `MIN_DOM_SCORE`: optional lower bound on domain bit score.
- `CPU`: threads to give HMMER.


In [ ]:
EVALUE_SEQ_MAX = 5e-4 #filters sequences based on their overall match probability to the HMM
EVALUE_DOM_MAX = 5e-4 #per-domain cutoff ensures that the detected bHLH region itself is statistically significant, even if the rest of the sequence aligns poorly
MIN_DOM_SCORE  = 50  # typical “trusted cutoffs” for Pfam domains range between 30–40 bits for marginal significance and >50 bits for strong matches
CPU = max(1, (os.cpu_count() or 2) // 2)
RUN_DIR = OUT / "hmm"; RUN_DIR.mkdir(exist_ok=True, parents=True)
STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
DOMTBL = RUN_DIR / f"PF00010_vs_ASC_targets.{STAMP}.domtblout"
LOG    = RUN_DIR / f"PF00010_vs_ASC_targets.{STAMP}.log"
print(DOMTBL)


/Users/gorkemdurmaz/Desktop/asc_project_10/results/hmm/PF00010_vs_ASC_targets.20251017_132511.domtblout


## Find HMMER binary (hmmsearch preferred)


In [28]:
from shutil import which

HMMSEARCH = which("hmmsearch")
HMMSCAN   = which("hmmscan")
print("hmmsearch:", HMMSEARCH)
print("hmmscan:", HMMSCAN)

if HMMSEARCH is None and HMMSCAN is None:
    raise SystemExit("Neither hmmsearch nor hmmscan found on PATH. Please install HMMER3.")


hmmsearch: /Users/gorkemdurmaz/miniconda3/envs/asc/bin/hmmsearch
hmmscan: /Users/gorkemdurmaz/miniconda3/envs/asc/bin/hmmscan


## Run scan
- If `hmmsearch` is available, search the PF00010 model against sequences.
- Else fall back to `hmmscan` (sequence DB is HMMs; here it is just a single PF00010 model).

In [29]:
cmd = None
if HMMSEARCH:
    cmd = [HMMSEARCH, "--domtblout", str(DOMTBL), "--cpu", str(CPU), str(HMM_PF00010), str(IN_TARGETS)]
    mode = "hmmsearch"
elif HMMSCAN:
    cmd = [HMMSCAN, "--domtblout", str(DOMTBL), "--cpu", str(CPU), str(HMM_PF00010), str(IN_TARGETS)]
    mode = "hmmscan"

print("Running:", " ".join(cmd))
with open(LOG, "w") as logf:
    proc = subprocess.run(cmd, stdout=logf, stderr=subprocess.STDOUT, check=False)
print("Return code:", proc.returncode)
assert DOMTBL.exists(), f"domtblout not written: {DOMTBL}"


Running: /Users/gorkemdurmaz/miniconda3/envs/asc/bin/hmmsearch --domtblout /Users/gorkemdurmaz/Desktop/asc_project_10/results/hmm/PF00010_vs_ASC_targets.20251017_132511.domtblout --cpu 7 /Users/gorkemdurmaz/Desktop/asc_project_10/data/PF00010.hmm /Users/gorkemdurmaz/Desktop/asc_project_10/data/ASC_targets.fasta
Return code: 0


## Parse domtblout → domain table
HMMER domtblout columns (subset used):
1. `tname` (target/sequence ID), 2. `tacc`, 3. `tlen`, 4. `qname` (PF00010), 6. `qlen`, 7. `E-value (full)`, 8. `full score`, 13. `i-Evalue (dom)`, 14. `dom score`, 15–22: positions (`hmm_from/to`, `ali_from/to`, `env_from/to`), 23 `acc`.


In [30]:
import pandas as pd
import math
import re

def _to_int(x):
    try:
        return int(x)
    except Exception:
        return None

def _to_float(x):
    try:
        if x == '-' or x is None:
            return math.nan
        return float(x)
    except Exception:
        return math.nan

def read_domtbl(path):
    # HMMER3 domtblout columns (per hmmer.org docs)
    # 0 tname, 1 tacc, 2 tlen, 3 qname, 4 qacc, 5 qlen,
    # 6 E-value, 7 score, 8 bias, 9 #, 10 of, 11 c-Evalue,
    # 12 i-Evalue, 13 score_dom, 14 bias_dom,
    # 15 hmm_from, 16 hmm_to, 17 ali_from, 18 ali_to,
    # 19 env_from, 20 env_to, 21 acc
    rows = []
    with open(path) as fh:
        for line in fh:
            if not line or line.startswith('#'):
                continue
            # split on any whitespace, but only take the first 22 fields
            parts = re.split(r'\s+', line.strip())
            if len(parts) < 22:
                # some lines can be truncated; skip safely
                continue
            parts = parts[:22]

            rows.append({
                'seq_id'     : parts[0],
                'seq_acc'    : None if parts[1] == '-' else parts[1],
                'seq_len'    : _to_int(parts[2]),
                'query'      : parts[3],
                'query_acc'  : None if parts[4] == '-' else parts[4],
                'qlen'       : _to_int(parts[5]),
                'e_full'     : _to_float(parts[6]),
                'score_full' : _to_float(parts[7]),
                'bias_full'  : _to_float(parts[8]),
                'hit_index'  : _to_int(parts[9]),      # domain number (#)
                'hit_of'     : _to_int(parts[10]),     # total domains (of)
                'c_evalue'   : _to_float(parts[11]),
                'i_evalue'   : _to_float(parts[12]),
                'score_dom'  : _to_float(parts[13]),
                'bias_dom'   : _to_float(parts[14]),
                'hmm_from'   : _to_int(parts[15]),
                'hmm_to'     : _to_int(parts[16]),
                'ali_from'   : _to_int(parts[17]),
                'ali_to'     : _to_int(parts[18]),
                'env_from'   : _to_int(parts[19]),
                'env_to'     : _to_int(parts[20]),
                'acc'        : _to_float(parts[21]),   # <-- correct index
            })
    return pd.DataFrame(rows)

df = read_domtbl(DOMTBL)
print(df.shape)
df.head()


(91, 22)


,seq_id,seq_acc,seq_len,query,query_acc,qlen,e_full,score_full,bias_full,hit_index,...,i_evalue,score_dom,bias_dom,hmm_from,hmm_to,ali_from,ali_to,env_from,env_to,acc
0,Hgra_g12400.t1,None,213,HLH,PF00010.31,53,6.600000e-23,72.9,0.6,1,...,1.300000e-22,72.0,0.6,3,53,77,127,75,127,0.97
1,Abru_g15100.t1,None,195,HLH,PF00010.31,53,3.800000e-22,70.5,0.5,1,...,6.300000e-22,69.8,0.5,3,53,76,126,74,126,0.97
2,Dsil_g4955.t1,None,248,HLH,PF00010.31,53,4.300000e-22,70.3,0.4,1,...,6.600000e-22,69.7,0.4,3,53,101,151,99,151,0.97
3,Ptep_aug3.g15679,None,238,HLH,PF00010.31,53,4.800000e-22,70.2,0.5,1,...,8.900000e-22,69.3,0.5,3,53,83,133,81,133,0.97
4,Ptep_aug3.g13882,None,210,HLH,PF00010.31,53,8.900000e-22,69.3,2.2,1,...,1.700000e-21,68.4,2.2,4,53,97,146,95,146,0.98


## Filter hits and compute anchors
- Keep domains where **full-seq E-value** ≤ `EVALUE_SEQ_MAX` and **domain i-Evalue** ≤ `EVALUE_DOM_MAX` and `score_dom` ≥ `MIN_DOM_SCORE`.
- Define the **anchor** as the integer center of the target alignment: `floor((ali_from + ali_to)/2)`.
- Keep multiple domains per sequence (bHLH duplications allowed).

In [31]:
f = df[(df['e_full'] <= EVALUE_SEQ_MAX) & (df['i_evalue'] <= EVALUE_DOM_MAX) & (df['score_dom'] >= MIN_DOM_SCORE)].copy()
f['dom_index'] = f.groupby('seq_id').cumcount() + 1
f['anchor_pos'] = ((f['ali_from'] + f['ali_to']) // 2).astype(int)
f.sort_values(['seq_id','dom_index'], inplace=True)
print(f.shape)
f.head()


(89, 24)


,seq_id,seq_acc,seq_len,query,query_acc,qlen,e_full,score_full,bias_full,hit_index,...,bias_dom,hmm_from,hmm_to,ali_from,ali_to,env_from,env_to,acc,dom_index,anchor_pos
54,Abru_g13702.t1,None,229,HLH,PF00010.31,53,2.200000e-20,64.8,3.4,1,...,3.4,4,53,119,168,117,168,0.98,1,143
61,Abru_g14616.t1,None,170,HLH,PF00010.31,53,3.800000e-20,64.1,0.6,1,...,0.6,4,53,60,110,58,110,0.96,1,85
67,Abru_g14798.t1,None,221,HLH,PF00010.31,53,6.400000e-20,63.4,0.1,1,...,0.1,3,53,84,152,82,152,0.97,1,118
65,Abru_g14799.t1,None,233,HLH,PF00010.31,53,6.300000e-20,63.4,1.0,1,...,0.2,3,53,93,143,91,143,0.97,1,118
48,Abru_g14800.t1,None,222,HLH,PF00010.31,53,1.400000e-20,65.4,0.8,1,...,0.8,3,53,112,162,110,162,0.97,1,137


## Exports
We write three outputs under `results/`:
1. `bHLH_anchors.tsv` — tidy table for downstream analysis.
2. `bHLH_anchors.gff3` — domains as GFF features (type=PF00010_bHLH).
3. `bHLH_anchors.json` — lightweight JSON for UI/plotting.


In [32]:
OUT_TSV  = OUT / "bHLH_anchors.tsv"
OUT_GFF  = OUT / "bHLH_anchors.gff3"
OUT_JSON = OUT / "bHLH_anchors.json"

cols = ['seq_id','seq_len','dom_index','anchor_pos','ali_from','ali_to','env_from','env_to','i_evalue','score_dom','e_full','score_full','acc','hmm_from','hmm_to']
f[cols].to_csv(OUT_TSV, sep='\t', index=False)
print("Wrote:", OUT_TSV)

with open(OUT_GFF, 'w') as gff:
    gff.write("##gff-version 3\n")
    for _, r in f.iterrows():
        attrs = [
            f"ID={r.seq_id}.PF00010.{r.dom_index}",
            f"Name=PF00010_bHLH",
            f"DomIndex={r.dom_index}",
            f"iE={r.i_evalue}",
            f"DomScore={r.score_dom}",
            f"Anchor={r.anchor_pos}"
        ]
        gff.write("\t".join([
            r.seq_id, "HMMER", "PF00010_bHLH",
            str(int(r.ali_from)), str(int(r.ali_to)),
            str(r.score_dom), ".", ".",
            ";".join(attrs)
        ]) + "\n")
print("Wrote:", OUT_GFF)

with open(OUT_JSON, 'w') as jf:
    jf.write(f.to_json(orient='records'))
print("Wrote:", OUT_JSON)


Wrote: /Users/gorkemdurmaz/Desktop/asc_project_10/results/bHLH_anchors.tsv
Wrote: /Users/gorkemdurmaz/Desktop/asc_project_10/results/bHLH_anchors.gff3
Wrote: /Users/gorkemdurmaz/Desktop/asc_project_10/results/bHLH_anchors.json


In [33]:
weak = df.query('score_dom < 60 or i_evalue > 1.6e-05')
weak.groupby(pd.cut(df['score_dom'], bins=[0,40,50,60,70,80,90,100])).size()


/var/folders/nj/mg_w6dbd39q6th2fvghj7bt00000gn/T/ipykernel_2859/2334998977.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weak.groupby(pd.cut(df['score_dom'], bins=[0,40,50,60,70,80,90,100])).size()


score_dom
(0, 40]      1
(40, 50]     1
(50, 60]     9
(60, 70]     0
(70, 80]     0
(80, 90]     0
(90, 100]    0
dtype: int64

## Quick sanity summaries
How many sequences/domains were detected; distribution of anchor positions, etc.

In [ ]:
print("Sequences with ≥1 bHLH:", f['seq_id'].nunique(), "of", df['seq_id'].nunique(), "total in domtblout")
dom_counts = f.groupby('seq_id')['dom_index'].max().describe()
display(dom_counts)

ax = f['anchor_pos'].plot(kind='hist', bins=40, title='Anchor positions (target AA index)')


## (Optional) write per-sequence anchor summary for MEME slicing later
This creates a simple `seq_id → list of (start,end,anchor)` JSON that i can reuse if decide to slice windows around bHLH (even though the main analysis will use full-length sequences).

In [ ]:
from collections import defaultdict
by_seq = defaultdict(list)
for _, r in f.iterrows():
    by_seq[r.seq_id].append({'start': int(r.ali_from), 'end': int(r.ali_to), 'anchor': int(r.anchor_pos), 'dom_index': int(r.dom_index)})
ANCHORS_JSON = OUT / "bHLH_anchors.by_seq.json"
import json
with open(ANCHORS_JSON, 'w') as jf:
    json.dump(by_seq, jf, indent=2)
print("Wrote:", ANCHORS_JSON)


### What next?
- Proceed to  MEME notebook to run **de‑novo motif discovery on full-length proteins**, optionally stratified by clade, now with reliable bHLH anchors for mapping/masking if needed.
- For tree overlays in iTOL/ETE3, the `bHLH_anchors.tsv` provides consistent coordinates and domain counts per protein.
